<a href="https://colab.research.google.com/github/JoaoVitttt/ED2_JJoaoVictor/blob/main/ED2_github_com_giiff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [73]:
class Vertice:
    '''
    Classe que representa um vertice tendo como atributos:
        - o valor do vertice que pode ser outro objeto qualquer (um número, uma string, etc...)
        - um conjunto de arestas incidentes ao vertice
    Criacao do objeto: v = Vertice(valor)
    '''
    def __init__(self, valor, arestas = set(), direcionado=True):
        '''
        Metodo construtor da classe
        '''
        self.__valor = valor
        self.__arestas = arestas
        self.__direcionado = direcionado
    
    def getValor(self):# retorna o valor do vertice
        return self.__valor
    
    def setValor(self, valor):# prove um valor ao vertice
        self.__valor = valor
        
    def getArestas(self): # retorna as arestas do vertice
        return self.__arestas
    
    def setAresta(self, aresta):# prove uma aresta ao vertice
        self.__arestas.add(aresta)
        
    def getArestasSaida(self): # retorna uma lista com as arestas que saem do vertice
        try:
            if self.__direcionado == False:
                return self.__arestas
            arestasDeSaida = []
            for aresta in self.__arestas:
                if aresta.getVerticeOrigem() == self:
                    arestasDeSaida.append(aresta)
            return arestasDeSaida
        except AttributeError: 
            return []
    
    def getArestasEntrada(self):# retorna uma lista com as arestas que entram do vertice
        try:
            if self.__direcionado == False:
                return self.__arestas
            arestasSaida = []
            for aresta in self.__arestas:
                if aresta.getVerticeDestino() == self:
                    arestasSaida.append(aresta)
            return arestasSaida
        except AttributeError: 
            return []
    
    def getGrau(self):# retorna o grau do vertice
        return len(self.getArestasSaida()) + len(self.getArestasEntrada())
    
    def getAdjacentes(self): # retorna uma lista com os vertices adjacentes ao vertice v
        listaVerticesAdjacentes = []
        for arestas_de_saida in self.getArestasSaida():
            listaVerticesAdjacentes.append(arestas_de_saida.getVerticeDestino())
        return listaVerticesAdjacentes

In [74]:
class Aresta:
    '''
    Classe que representa uma aresta tendo como atributos:
        - vertice_origem: o vertice de origem (v:Vertice)
        - vertice_destino: o vertice de destino (v:vertice)
        - peso: o peso da aresta que pode ser um objeto qualquer (int, str, etc...)
        - direcionada: flag indicando de a aresta e direcionado ou não (padrao True)
    '''
    def __init__(self, vertice_origem:Vertice, vertice_destino:Vertice, peso = 1, direcionada = True):
        '''
        Metodo construtor da classe: preenche os atributos
        '''
        self.__vertice_origem = vertice_origem
        self.__vertice_destino = vertice_destino
        self.__peso = peso
        self.__direcionada = direcionada
        self.__vertice_origem.setAresta(self)
        self.__vertice_destino.setAresta(self)
        
    def getVerticeOrigem(self):
        return self.__vertice_origem
    def getVerticeDestino(self):
        return self.__vertice_destino
    def getPeso(self):
        return self.__peso
    

In [75]:
from collections import deque
import sys

class Grafo:
    '''
    Classe que representa o grafo tendo como atributos:
        - um conjunto de vertices nao vazio
        - um conjunto de arestas (vazio ou nao)
    '''
    def __init__(self, x):
    #def __init__(self, x, i, f):
        '''
        Metodo construtor da classe: Tarefas
        '''
        #self.Vertices = x
        #self.inicio = i
        #self.fim = f
        #self.peso = f - i
        '''
        Metodo construtor da classe: Dijkstra, BellmanFord, Prim e Kruskal
        '''
        self.Vertices = x
        self.Grafo = [] #[[0 for column in range(x)]for row in range(x)]
        '''
        Metodo construtor da classe: preenche os atributos
        '''
        self.__vertices = set()
        self.__arestas  = set()
        
    def setVertices(self, vertices):# prove o conjunto de vertices do grafo
        self.__vertices = vertices

    def getVertices(self):# retorna o conjunto de vertices do grafo
        return self.__vertices
        
    def setArestas(self, arestas):# prove o conjunto de arestas do grafo
        self.__arestas = arestas
        
    def getArestas(self):# retorna o conjunto de arestas do grafo
        return self.__arestas
    
    def getVerticeByValor(self, valor):# retorna um objeto da classe Vertice contendo um valor informado, se existir
        for v in self.__vertices:
            if v.getValor() == valor:
                return v
        return None

    def adicionarVertice(self, valor):# Adiciona um vertice ao grafo, caso nao exista vertice com o mesmo valor
        if self.buscarPorValor(valor) != valor:# valor nao esta no grafo
            self.__vertices.add(Vertice(valor))
            return True
        return False
    
    def adicionarAresta(self, origem, destino, peso = 1, direcionada = True):# adicionado uma aresta ao grafo ligando 2 vertices
        try:
            verticeOrigem = self.getVerticeByValor(origem)
            verticeDestino = self.getVerticeByValor(destino)
            if (verticeOrigem or verticeDestino) is None:  # existem os vertices de origem e destino?
                print("Nao ha no grafo, vertices de origem ou de destino com os valores informados.")
            self.getArestas().add(Aresta(verticeOrigem, verticeDestino, peso, direcionada))
        except AttributeError as error:
            print("Exceção: Nao ha no grafo, vertices de origem ou de destino com os valores informados.")
    
    def checkHandShakingLemma(self):# checa o lema do "aperto de mao"
        somaGraus = 0
        for v in self.getVertices():
            somaGraus+= v.getGrau()
        if somaGraus == len(self.getArestas())*2:
            return True
        else:
            return False
        
    def dfs(self, graph, v, visitados=[]):# busca em profundidade recursiva
        if v not in visitados: # se v nao foi visitado
            visitados.append(v) # marca vertice como visitado
        if len(v.getAdjacentes()) == 0: # vertice escolhido nao tem adjacentes
            self.dfs(graph, next(iter(graph.getVertices())), visitados) # chamada recursiva pegando o proximo vertice do set
        else: # vertice escolhido tem adjacentes
            for adjacente in v.getAdjacentes(): #percorre todos os adjacentes a ele
                if adjacente not in visitados: # se um dos adjacentes nao foi visitado
                    self.dfs(graph, adjacente, visitados) # chamada recursiva para cada adjacente
        return visitados
    
    def bfs(self, v, visitados = [], fila = deque([])):# busca em largura recursiva
        fila.append(v)  # adiciona o vertice v a fila
        if v not in visitados:  # se vertice v nao esta em visitados
            visitados.append(v)  # adiciona vertice v a visitados
        while fila:  # enquanto houver vertices na fila
            vertice = fila.popleft()  # tira vertice ja visitado da fila
            if len(vertice.getArestasSaida()) == 0: # vertice escolhido nao tem adjacentes
                self.bfs(next(iter(self.getVertices())), visitados, fila) # chamada recursiva pegando o proximo vertice do set   
            else:
                for adjacente in vertice.getAdjacentes(): #percorre todos os adjacentes a ele
                    if adjacente not in visitados: # se um dos adjacentes nao foi visitado
                        visitados.append(adjacente)  # insere o adjacente em visitados
                        self.bfs(adjacente, visitados, fila) # chamada recursiva pegando o proximo vertice do set
        return visitados  # retorna a lista de visitados  
    
    def buscarPorValor(self, valor):# busca um valor no grafo usando a busca em profundidade O(|V|+|E|)
        for v in self.bfs(next(iter(self.getVertices())), visitados = [], fila = deque([])):
            if valor == v.getValor():
                return valor
        return None
    
    def existeCaminhoEuler(self):# checa a existencia de um caminho Euleriano no grafo
        pares = True
        impares = []
        for v in self.getVertices():
            if v.getGrau() % 2 != 0:
                pares = False
                impares.append(v)
        return pares or len(impares) == 2
    
    def removerAresta(self, origem, destino, peso = 1): # remove uma aresta do grafo
        try:
            verticeOrigem  = self.getVerticeByValor(origem)
            verticeDestino = self.getVerticeByValor(destino)
            if (verticeOrigem or verticeDestino) is None:  # existem os vertices de origem e destino?
                pass
            else:
                arestas_a_remover = []
                for aresta in self.getArestas():
                    if aresta.getVerticeOrigem() == verticeOrigem and aresta.getVerticeDestino() == verticeDestino and aresta.getPeso() == peso:
                        arestas_a_remover.append(aresta)
                [self.getArestas().remove(aresta) for aresta in arestas_a_remover]
        except AttributeError as error:
            print("Exceção: Nao ha no grafo, vertices de origem ou de destino com os valores informado!.")
            
    def removerVertice(self, valor):# remove um vertice do grafo
        try:
            vertice  = self.getVerticeByValor(valor)
            if vertice is None:  # existem os vertices de origem e destino?
                pass
            else:
                arestas_a_remover = []
                # arestas conectadas ao vertice precisam ser removidas antes
                for a in vertice.getArestasSaida():
                    arestas_a_remover.append(a)
                for a in vertice.getArestasEntrada():
                    arestas_a_remover.append(a)
                [self.removerAresta(a.getVerticeOrigem(), a.getVerticeDestino(), a.getPeso()) for a in arestas_a_remover]    
                self.__vertices.remove(vertice)
        except AttributeError as error:
            print(repr(error))
            print("Exceção: Nao ha no grafo, vertices de origem ou de destino com os valores informado.")
            
            
    def getMatrizAdjacencia(self):# retorna uma matriz de adjacencia como dataframe pandas
        import numpy as np
        import pandas as pd
        V = self.getVertices()
        E = self.getArestas()
        matriz = np.zeros((len(V), len(V)), dtype=object)
        columns = []
        [columns.append(v.getValor()) for v in V]
        index =[]
        [index.append(v.getValor()) for v in V]
        matrizDeAdjacencias = pd.DataFrame(matriz, columns = columns, index = index)
        for index, row in matrizDeAdjacencias.iterrows():
            for e in E:
                matrizDeAdjacencias.loc[e.getVerticeOrigem().getValor(), e.getVerticeDestino().getValor()] = e.getPeso()
        return matrizDeAdjacencias
    
    def getMatrizAdjacenciaAsArray(self):# converte a matriz de adjacencia pandas.dataframe para array
        return self.getMatrizAdjacencia().to_numpy()
    
    def getMatrizAdjacenciaAsDict(self):# converte a matriz de adjacencia pandas.dataframe para dicionario (lista de adjacencia)
        return self.getMatrizAdjacencia().to_dict('dict')
    
    def WD40(self, distancia):
        print("Distância do vértice (Dijkstra):")
        for n in range(self.Vertices):
            print(n, "-->", distancia[n])

    def minDistancia(self, distancia, swDij):
        min = sys.maxsize
        for v in range(self.Vertices):
            if distancia[v] < min and swDij[v] == False:
                min = distancia[v]
                min_index = v
        return min_index
 
    def Dijkstra(self, value):
        distancia = [sys.maxsize] * self.Vertices
        distancia[value] = 0
        swDij = [False] * self.Vertices
        for cout in range(self.Vertices):
            a = self.minDistancia(distancia, swDij)
            swDij[a] = True
            for v in range(self.Vertices):
                if self.grafo[a][v] > 0 and swDij[v] == False and distancia[v] > distancia[a] + self.grafo[a][v]: distancia[v] = distancia[a] + self.grafo[a][v]

        self.WD40(distancia)

    def add(self, a, b, c):
        self.Grafo.append([a, b, c])

    def solucao(self, distancia):
        print("Distância do vértice (BellmanFord):")
        for a in range(self.Vertices):
            print("{0} --> {1}".format(a, distancia[a]))

    def BellmanFord(self, value):
        distancia = [float("Inf")] * self.Vertices
        distancia[value] = 0
        for _ in range(self.Vertices - 1):
            for a, b, c in self.Grafo:
                if distancia[a] != float("Inf") and distancia[a] + c < distancia[b]:
                    distancia[b] = distancia[a] + c
        for a, b, c in self.Grafo:
            if distancia[a] != float("Inf") and distancia[a] + c < distancia[b]:
                print("Grafo negativo")
                return

        self.solucao(distancia)
    
    def printMSG(self, parent):
        print("Árvore geradora mínima (Prim):")
        for i in range(1, self.Vertices):
            print(parent[i], "-", i, "-->", self.grafo[i][parent[i]])

    def minKey(self, key, primSet):
        min = sys.maxsize
        for v in range(self.Vertices):
            if key[v] < min and primSet[v] == False:
                min = key[v]
                min_index = v
        return min_index

    def Prim(self):
        key = [sys.maxsize] * self.Vertices
        parent = [None] * self.Vertices 
        key[0] = 0
        primSet = [False] * self.Vertices
        parent[0] = -1 
        for cout in range(self.Vertices):
            u = self.minKey(key, primSet)
            primSet[u] = True
            for v in range(self.Vertices):
                if self.grafo[u][v] > 0 and primSet[v] == False and key[v] > self.grafo[u][v]:
                        key[v] = self.grafo[u][v]
                        parent[v] = u
 
        self.printMSG(parent)
    
    def search(self, parent, i):
        if parent[i] == i:
            return i
        return self.search(parent, parent[i])
 
    def union(self, parent, rank, x, y):
        x = self.search(parent, x)
        y = self.search(parent, y)
        if rank[x] < rank[y]:
            parent[x] = y
        elif rank[x] > rank[y]:
            parent[y] = x
        else:
            parent[y] = x
            rank[x] += 1
 
    def kruskal(self):
        resto = []
        i, e = 0, 0
        self.Grafo = sorted(self.Grafo, key=lambda item: item[2])
        parent = []
        rank = []
        for node in range(self.Vertices):
            parent.append(node)
            rank.append(0)
        while e < self.Vertices - 1:
            a, b, c = self.Grafo[i]
            i = i + 1
            x = self.search(parent, a)
            y = self.search(parent, b)
            if x != y:
                e = e + 1
                resto.append([a, b, c])
                self.union(parent, rank, x, y)
        for a, b, weight in resto:
            print(a,"-", b, end = " ")
            print("-->",weight)
            

In [76]:
# Algoritmo guloso - Número Máximo de Tarefas:

from collections import deque

class Grafo:

  def __init__(self, x, i ,f):
    '''
    Metodo construtor da classe: Tarefas
    '''
    self.Vertices = x
    self.inicio = i
    self.fim = f
    self.peso = f - i

#Esclher o que termina primeiro:
o1 = Grafo("T1", 0, 2)
o2 = Grafo("T2", 1, 3)
o3 = Grafo("T3", 2, 6)
o4 = Grafo("T4", 4, 6)
o5 = Grafo("T5", 4, 7)
o6 = Grafo("T6", 7, 8)
o7 = Grafo("T7", 6, 10)
o8 = Grafo("T8", 8, 11)

lista=[o1, o2, o3, o4, o5, o6, o7, o8]

lista.sort(key=lambda t: t.fim)
for item in lista:
  print(item.fim, "-->", item.Vertices)
solucao=[]
solucao.append(lista[0])
posicao = 0

for i in range(1, len(lista), +1):
  if lista[i].inicio >= lista[posicao].fim:
    solucao.append(lista[i])
    pos = i
print("A quantidade máxima de tarefas é: ", len(solucao))



2 --> T1
3 --> T2
6 --> T3
6 --> T4
7 --> T5
8 --> T6
10 --> T7
11 --> T8
A quantidade máxima de tarefas é:  7


In [77]:
#Esclher o que começa primeiro:
o1 = Grafo("T1", 0, 2)
o2 = Grafo("T2", 1, 3)
o3 = Grafo("T3", 2, 6)
o4 = Grafo("T4", 4, 6)
o5 = Grafo("T5", 4, 7)
o6 = Grafo("T6", 7, 8)
o7 = Grafo("T7", 6, 10)
o8 = Grafo("T8", 8, 11)

lista=[o1, o2, o3, o4, o5, o6, o7, o8]

lista.sort(key=lambda t: t.inicio)
for item in lista:
  print(item.fim, "-->", item.Vertices)
solucao=[]
solucao.append(lista[0])
posicao = 0

for i in range(1, len(lista), +1):
  if lista[i].inicio >= lista[posicao].fim:
    solucao.append(lista[i])
    pos = i
print("A quantidade máxima de tarefas é: ", len(solucao))

2 --> T1
3 --> T2
6 --> T3
6 --> T4
7 --> T5
10 --> T7
8 --> T6
11 --> T8
A quantidade máxima de tarefas é:  7


In [78]:
#Esclher o de menor duração:
o1 = Grafo("T1", 0, 2)
o2 = Grafo("T2", 1, 3)
o3 = Grafo("T3", 2, 6)
o4 = Grafo("T4", 4, 6)
o5 = Grafo("T5", 4, 7)
o6 = Grafo("T6", 7, 8)
o7 = Grafo("T7", 6, 10)
o8 = Grafo("T8", 8, 11)

lista=[o1, o2, o3, o4, o5, o6, o7, o8]

lista.sort(key=lambda t: t.peso)
for item in lista:
  print(item.fim, "-->", item.Vertices)
solucao=[]
solucao.append(lista[0])
posicao = 0

for i in range(1, len(lista), +1):
  if lista[i].inicio >= lista[posicao].fim:
    solucao.append(lista[i])
    pos = i
print("A quantidade máxima de tarefas é: ", len(solucao))

8 --> T6
2 --> T1
3 --> T2
6 --> T4
7 --> T5
11 --> T8
6 --> T3
10 --> T7
A quantidade máxima de tarefas é:  2


In [60]:
# Problema da Mochila
from typing import List

class Grafo:
  def __init__(self, x, p ,v):
    self.Vertices = x
    self.valor = v
    self.peso = p

o1 = Grafo("Ouro", 2, 100)
o2 = Grafo("Prata", 3, 50)
o3 = Grafo("Bronze", 2, 30)
o4 = Grafo("Cobre", 6, 120)
o5 = Grafo("Platina", 1, 20)

lista=[o1, o2, o3, o4, o5]

  def Mochila(self, n, item, cap, lista = []):
    lista.append(item)
    if len(n or cap) == 0:
      return 0
    if len(item(n-1), self.peso() > cap):
      return item(n-1)
    else:
      usa = self.Mochila(item(n-1), lista, cap-self.peso())
      nusa = self.Mochila(item(n-1), lista, cap)
  print(Mochila(usa, "-->", nusa))
  print("O melhor lucro é de: ", len(self.Vertices))

'''
public static int mochila(int n, List<Item> itens, int cap){
    if(n==0 || cap==0){
        return 0;
    }
    if(itens.get(n-1).getPeso() > cap){
        return mochila(n-1, itens, cap);
    }
    else{
        return Math.max(itens.get(n-1).getValor() + mochila(n-1, itens, cap-itens.get(n-1).getPeso()),
                        mochila(n-1, itens, cap));
    }
}
'''

In [ ]:
def recursivoMenorValor(listaNotas, valor):
  menorValor = valor
  if valor in listaNotas:
    return 1
  else:
    for i in [c for c in listaNotas if c <= valor]:
      quantNotas = 1 + recursivoMenorValor(listaNotas, valor-1)
      if quantNotas < menorValor:
        menorValor =  quantNotas
  return menorValor

print(recursivoMenorValor([1, 2, 5], 35))

In [21]:
print("Árvore geradora mínima (Kruskal):")
G = Grafo(5)
G.add(0, 1, 8)
G.add(0, 2, 5)
G.add(1, 2, 9)
G.add(1, 3, 11)
G.add(2, 3, 15)
G.add(2, 4, 10)
G.add(3, 4, 7)
G.kruskal()

Árvore geradora mínima (Kruskal):
0 - 2 --> 5
3 - 4 --> 7
0 - 1 --> 8
2 - 4 --> 10


In [45]:
#("Árvore geradora mínima (Prim):")
G = Grafo(5)
G.grafo = [ [0, 2, 0, 6, 0],
            [2, 0, 3, 8, 5],
            [0, 3, 0, 0, 7],
            [6, 8, 0, 0, 9],
            [0, 5, 7, 9, 0]]
 
G.Prim();

Árvore geradora mínima (Prim):
0 - 1 --> 2
1 - 2 --> 3
0 - 3 --> 6
1 - 4 --> 5


In [58]:
# Distância do vértice (Dijkstra):
G = Grafo(9)
G.grafo = [[0, 4, 0, 0, 0, 0, 0, 8, 0],
           [4, 0, 8, 0, 0, 0, 0, 11, 0],
           [0, 8, 0, 7, 0, 4, 0, 0, 2],
           [0, 0, 7, 0, 9, 14, 0, 0, 0],
           [0, 0, 0, 9, 0, 10, 0, 0, 0],
           [0, 0, 4, 14, 10, 0, 2, 0, 0],
           [0, 0, 0, 0, 0, 2, 0, 1, 6],
           [8, 11, 0, 0, 0, 0, 1, 0, 7],
           [0, 0, 2, 0, 0, 0, 6, 7, 0]
           ]

G.Dijkstra(0)

Distância do vértice (Dijkstra):
0 --> 0
1 --> 4
2 --> 12
3 --> 19
4 --> 21
5 --> 11
6 --> 9
7 --> 8
8 --> 14


In [ ]:
#Distância do vértice (BellmanFord):
G = Grafo(5)
G.add(0, 1, 2)
G.add(0, 2, 4)
G.add(1, 3, 2)
G.add(2, 4, 3)
G.add(2, 3, 4)
G.add(4, 3, -5)

G.BellmanFord(0)

Distância do vértice (BellmanFord):
0 --> 0
1 --> 2
2 --> 4
3 --> 2
4 --> 7


In [ ]:
v1 = Vertice(1)
v2 = Vertice(2)
v3 = Vertice(3)
v4 = Vertice(4)
v5 = Vertice(5)
a1 = Aresta( v1, v2, 10, True )
a2 = Aresta( v2, v3, 20, True )
a3 = Aresta( v3, v4, 30, True )
a4 = Aresta( v4, v1, 40, True )
a5 = Aresta( v4, v5, 50, True )

In [ ]:
G = Grafo(5)
G.setVertices({v1, v2, v3, v4, v5})
G.setArestas({a1, a2, a3, a4, a5})

In [ ]:
for v in G.getVertices():
    print(v.getValor(), end="\t")

2	3	5	4	1	

In [ ]:
for a in G.getArestas():
    print(a.getVerticeOrigem().getValor(), end="")
    print(" --", a.getPeso(), "--> ", end="")
    print(a.getVerticeDestino().getValor())

1 -- 10 --> 2
2 -- 20 --> 3
3 -- 30 --> 4
4 -- 40 --> 1
4 -- 50 --> 5


In [ ]:
G.getMatrizAdjacencia()

,2,3,5,4,1
2,0,20,0,0,0
3,0,0,0,30,0
5,0,0,0,0,0
4,0,0,50,0,40
1,10,0,0,0,0


In [ ]:
G.getMatrizAdjacenciaAsArray()

array([[0, 20, 0, 0, 0],
       [0, 0, 0, 30, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 50, 0, 40],
       [10, 0, 0, 0, 0]], dtype=object)

In [ ]:
G.getMatrizAdjacenciaAsDict()

{1: {1: 0, 2: 0, 3: 0, 4: 40, 5: 0},
 2: {1: 10, 2: 0, 3: 0, 4: 0, 5: 0},
 3: {1: 0, 2: 20, 3: 0, 4: 0, 5: 0},
 4: {1: 0, 2: 0, 3: 30, 4: 0, 5: 0},
 5: {1: 0, 2: 0, 3: 0, 4: 50, 5: 0}}

In [ ]:
v1.getGrau()

2

In [ ]:
G.checkHandShakingLemma()

True

In [ ]:
G.removerAresta(1, 2, peso = 10)
for a in G.getArestas():
    print(a.getVerticeOrigem().getValor(), end="")
    print(" --", a.getPeso(), "--> ", end="")
    print(a.getVerticeDestino().getValor())

2 -- 20 --> 3
3 -- 30 --> 4
4 -- 40 --> 1
4 -- 50 --> 5


In [ ]:
# Remocao de vertice
print("Antes de remover")
for v in G.getVertices():
    print(v.getValor(), end="\t")
    
G.removerVertice(1)
print("\nDepois de remover")
for v in G.getVertices():
    print(v.getValor(), end="\t")

Antes de remover
2	3	5	4	1	
Depois de remover
2	3	5	4	

In [ ]:
#Busca em profundidade:
v1 = Vertice(1)
v2 = Vertice(2)
v3 = Vertice(3)
v4 = Vertice(4)
v5 = Vertice(5)
v6 = Vertice(6)
a1 = Aresta( v1, v2, 10, True )
a2 = Aresta( v2, v3, 20, True )
a3 = Aresta( v3, v4, 30, True )
a4 = Aresta( v4, v1, 40, True )
a5 = Aresta( v4, v5, 50, True ) 
a6 = Aresta( v4, v6, 60, True ) 
G = Grafo(6)
G.setVertices({v1, v2, v3, v4, v5,v6})
G.setArestas({a1, a2, a3, a4, a5, a6})
for vertice in G.getVertices():
    print(f"Busca em profundidade, iniciando com o vértice {vertice.getValor()}:")
    for v in G.dfs(G, vertice, visitados=[]):
        print(str(v.getValor())+"\t", end="")
    print("\n.................................................") 

Busca em profundidade, iniciando com o vértice 2:
2	3	4	6	5	1	
.................................................
Busca em profundidade, iniciando com o vértice 1:
1	2	3	4	6	5	
.................................................
Busca em profundidade, iniciando com o vértice 3:
3	4	6	2	5	1	
.................................................
Busca em profundidade, iniciando com o vértice 4:
4	6	2	3	5	1	
.................................................
Busca em profundidade, iniciando com o vértice 5:
5	2	3	4	6	1	
.................................................
Busca em profundidade, iniciando com o vértice 6:
6	2	3	4	5	1	
.................................................


In [ ]:
#Busca em largura:
v1 = Vertice(1)
v2 = Vertice(2)
v3 = Vertice(3)
v4 = Vertice(4)
v5 = Vertice(5)
v6 = Vertice(6)
a1 = Aresta( v1, v2, 10, True )
a2 = Aresta( v2, v3, 20, True )
a3 = Aresta( v3, v4, 30, True )
a4 = Aresta( v4, v1, 40, True )
a5 = Aresta( v4, v5, 50, True ) 
a6 = Aresta( v4, v6, 60, True ) 
G = Grafo(6)
G.setVertices({v1, v2, v3, v4, v5,v6})
G.setArestas({a1, a2, a3, a4, a5, a6})
for vertice in G.getVertices():
    print(f"Busca em largura, iniciando com o vértice {vertice.getValor()}:")
    for v in  G.bfs(vertice, visitados = [], fila = deque([])):
        print(str(v.getValor())+"\t", end="")
    print("\n............................................")   

Busca em largura, iniciando com o vértice 1:
1	2	3	4	5	6	
............................................
Busca em largura, iniciando com o vértice 3:
3	4	5	1	2	6	
............................................
Busca em largura, iniciando com o vértice 4:
4	5	1	2	3	6	
............................................
Busca em largura, iniciando com o vértice 5:
5	1	2	3	4	6	
............................................
Busca em largura, iniciando com o vértice 6:
6	1	2	3	4	5	
............................................
Busca em largura, iniciando com o vértice 2:
2	3	4	5	1	6	
............................................


In [ ]:
valor = int(input("Valor procurado: "))
if G.buscarPorValor(valor) == None:
    print(f"{valor} não encontrado no grafo")
else:
    print(f"{valor} encontrado no grafo")
# [print(f"{v.getValor()} encontrado no grafo.") for v in  G.bfs(v1, visitados = [], fila = deque([])) if n == v.getValor() ]

Valor procurado: 144
144 não encontrado no grafo


In [ ]:
if G.existeCaminhoEuler():
    print("Grafo contém um caminho Euleriano")
else:
    print("Grafo NÃO contém um caminho Euleriano")

Grafo contém um caminho Euleriano
